In [1]:
import numpy as np
import pandas as pd
import random

from pathlib import Path
from tqdm import tqdm

import tensorflow as tf
from tensorflow.keras.metrics import AUC
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Activation, Conv1D, Dense, GlobalAveragePooling1D, GRU, Input, BatchNormalization, Dropout
from ncps.wirings import AutoNCP
from ncps.keras import LTC

2025-05-03 01:30:47.301225: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-03 01:30:47.314248: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-03 01:30:47.340971: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746225047.382905   42534 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746225047.393442   42534 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-03 01:30:47.435569: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

# Configuration

In [2]:
NUM_EPOCHS = 400
NUM_EXPERIMENTS = 5


def create_model(train):
    model = Sequential()
    model.add(Input(shape=(train.shape[1], train.shape[2])))

    model.add(LTC(AutoNCP(64, 32), return_sequences=True))
    model.add(LTC(AutoNCP(32, 16), return_sequences=False))

    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer=Adam(learning_rate=0.002, weight_decay=1e-7, use_ema=True), loss='binary_crossentropy', metrics=["accuracy", AUC(name="auc")])
    return model

# Experiment

In [3]:
ID = ["ID"]
USER = ["SubjectID"]
IDS = ["SubjectID", "VideoID"]
TARGET = ["predefinedlabel"]
FEATURES = ["Delta", "Theta", "Alpha1", "Alpha2", "Beta1", "Beta2", "Gamma1", "Gamma2"]
INIT_SEED = 5412

In [4]:
data_dir = Path("/home/aseliverstov/projects/brain_signals/data")
data = pd.read_csv(data_dir / "EEG_data.csv")

data["ID"] = (len(np.unique(data["VideoID"])) * data["SubjectID"] + data["VideoID"]).astype("int")
data = data[ID + USER + FEATURES + TARGET]

data.head(3)

,ID,SubjectID,Delta,Theta,Alpha1,Alpha2,Beta1,Beta2,Gamma1,Gamma2,predefinedlabel
0,0,0.0,301963.0,90612.0,33735.0,23991.0,27946.0,45097.0,33228.0,8293.0,0.0
1,0,0.0,73787.0,28083.0,1439.0,2240.0,2746.0,3687.0,5293.0,2740.0,0.0
2,0,0.0,758353.0,383745.0,201999.0,62107.0,36293.0,130536.0,57243.0,25354.0,0.0


In [5]:
def reshape_dataset(data):
    features = []
    target = []
    for cur_id in np.unique(data[ID].to_numpy()):
        cur_id_data = data[data[ID].to_numpy() == cur_id]
        target.append(np.mean(cur_id_data[TARGET].to_numpy()).astype("int"))
        features.append(cur_id_data[FEATURES].to_numpy())

    features = pad_sequences(features)
    return np.array(features), np.array(target)

def pad_sequences(arrays, pad_value=0):
    max_length = max(arr.shape[0] for arr in arrays)
    padded_arrays = [
        np.pad(
            arr,
            ((0, max_length - arr.shape[0]), (0, 0)),
            mode='constant',
            constant_values=pad_value)
            for arr in arrays
        ]
    return np.stack(padded_arrays)

In [6]:
X, _ = reshape_dataset(data)
model = create_model(X)
model.summary()

2025-05-03 01:30:51.854883: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ ltc (LTC)                       │ (None, 144, 32)        │        18,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ ltc_1 (LTC)                     │ (None, 16)             │         8,384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 27,105 (105.88 KB)

 Trainable params: 27,105 (105.88 KB)

 Non-trainable params: 0 (0.00 B)

In [7]:
%%time

all_acc = []
all_loss = []
all_auc = []

all_val_acc = []
all_val_loss = []
all_val_auc = []

for j, seed in tqdm(enumerate(np.arange(NUM_EXPERIMENTS) + INIT_SEED)):
    np.random.seed(int(seed))
    random.seed(int(seed))
    tf.random.set_seed(int(seed))

    train_id = np.random.choice(np.unique(np.ravel(data[USER])), 7, replace=False)
    train_index = np.isin(data[USER], train_id)

    train = data.iloc[train_index]
    test = data.iloc[~train_index]

    X_train, y_train = reshape_dataset(train)
    X_test, y_test = reshape_dataset(test)

    y_train = y_train.reshape(-1, 1)
    y_test = y_test.reshape(-1, 1)

    model = create_model(X_train)

    history = model.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=NUM_EPOCHS,
        batch_size=10,
        verbose=1,
    )

    acc = history.history['accuracy']
    loss = history.history['loss']
    auc = history.history['auc']

    val_acc = history.history['val_accuracy']
    val_loss = history.history['val_loss']
    val_auc = history.history['val_auc']

    all_acc.append(acc)
    all_loss.append(loss)
    all_auc.append(auc)

    all_val_acc.append(val_acc)
    all_val_loss.append(val_loss)
    all_val_auc.append(val_auc)

epoch_acc = np.mean(all_acc, axis=0)
epoch_loss = np.mean(all_loss, axis=0)
epoch_auc = np.mean(all_auc, axis=0)

epoch_val_acc = np.mean(all_val_acc, axis=0)
epoch_val_loss = np.mean(all_val_loss, axis=0)
epoch_val_auc = np.mean(all_val_auc, axis=0)

0it [00:00, ?it/s]

Epoch 1/400
7/7 ━━━━━━━━━━━━━━━━━━━━ 26s 2s/step - accuracy: 0.4468 - auc: 0.4481 - loss: 0.6974 - val_accuracy: 0.5000 - val_auc: 0.8000 - val_loss: 0.6918
Epoch 2/400
7/7 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.5335 - auc: 0.6541 - loss: 0.6880 - val_accuracy: 0.5000 - val_auc: 0.8222 - val_loss: 0.6847
Epoch 3/400
7/7 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.6191 - auc: 0.7003 - loss: 0.6834 - val_accuracy: 0.8333 - val_auc: 0.7622 - val_loss: 0.6736
Epoch 4/400
7/7 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.7615 - auc: 0.7176 - loss: 0.6744 - val_accuracy: 0.8333 - val_auc: 0.8222 - val_loss: 0.6544
Epoch 5/400
7/7 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.7615 - auc: 0.7282 - loss: 0.6577 - val_accuracy: 0.8333 - val_auc: 0.8222 - val_loss: 0.6254
Epoch 6/400
7/7 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.7615 - auc: 0.7437 - loss: 0.6344 - val_accuracy: 0.8333 - val_auc: 0.7489 - val_loss: 0.5887
Epoch 7/400
7/7 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 

1it [1:01:07, 3667.48s/it]

Epoch 1/400
7/7 ━━━━━━━━━━━━━━━━━━━━ 35s 2s/step - accuracy: 0.4506 - auc: 0.4663 - loss: 0.7210 - val_accuracy: 0.5000 - val_auc: 0.5000 - val_loss: 0.6941
Epoch 2/400
7/7 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - accuracy: 0.4435 - auc: 0.4203 - loss: 0.6992 - val_accuracy: 0.5000 - val_auc: 0.5000 - val_loss: 0.6925
Epoch 3/400
7/7 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 0.5494 - auc: 0.4470 - loss: 0.6929 - val_accuracy: 0.5000 - val_auc: 0.8000 - val_loss: 0.6902
Epoch 4/400
7/7 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 0.5919 - auc: 0.7609 - loss: 0.6885 - val_accuracy: 0.8000 - val_auc: 0.7333 - val_loss: 0.6827
Epoch 5/400
7/7 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 0.7667 - auc: 0.7653 - loss: 0.6826 - val_accuracy: 0.8000 - val_auc: 0.6800 - val_loss: 0.6720
Epoch 6/400
7/7 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 0.7667 - auc: 0.7235 - loss: 0.6755 - val_accuracy: 0.8000 - val_auc: 0.7600 - val_loss: 0.6612
Epoch 7/400
7/7 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accurac

2it [2:26:51, 4536.13s/it]

...

Epoch 1/400
7/7 ━━━━━━━━━━━━━━━━━━━━ 36s 2s/step - accuracy: 0.5619 - auc: 0.7332 - loss: 0.6830 - val_accuracy: 0.7667 - val_auc: 0.8111 - val_loss: 0.6659
Epoch 2/400
7/7 ━━━━━━━━━━━━━━━━━━━━ 14s 2s/step - accuracy: 0.8038 - auc: 0.7994 - loss: 0.6595 - val_accuracy: 0.8333 - val_auc: 0.7667 - val_loss: 0.6326
Epoch 3/400
7/7 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - accuracy: 0.8038 - auc: 0.7926 - loss: 0.6272 - val_accuracy: 0.8333 - val_auc: 0.7667 - val_loss: 0.5922
Epoch 4/400
7/7 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - accuracy: 0.8038 - auc: 0.7871 - loss: 0.5876 - val_accuracy: 0.8333 - val_auc: 0.7667 - val_loss: 0.5458
Epoch 5/400
7/7 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - accuracy: 0.8038 - auc: 0.7846 - loss: 0.5423 - val_accuracy: 0.8333 - val_auc: 0.8111 - val_loss: 0.4978
Epoch 6/400
7/7 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - accuracy: 0.8038 - auc: 0.7846 - loss: 0.5023 - val_accuracy: 0.8333 - val_auc: 0.7667 - val_loss: 0.4696
Epoch 7/400
7/7 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - accurac

4it [5:17:08, 4921.31s/it]

Epoch 1/400
7/7 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - accuracy: 0.6401 - auc: 0.7776 - loss: 0.6583 - val_accuracy: 0.8000 - val_auc: 0.7733 - val_loss: 0.6218
Epoch 2/400
7/7 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.8754 - auc: 0.8467 - loss: 0.5854 - val_accuracy: 0.8000 - val_auc: 0.7467 - val_loss: 0.5761
Epoch 3/400
7/7 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - accuracy: 0.8754 - auc: 0.8136 - loss: 0.5351 - val_accuracy: 0.8000 - val_auc: 0.6800 - val_loss: 0.5392
Epoch 4/400
7/7 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - accuracy: 0.8754 - auc: 0.8115 - loss: 0.4848 - val_accuracy: 0.8000 - val_auc: 0.7467 - val_loss: 0.5130
Epoch 5/400
7/7 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - accuracy: 0.8754 - auc: 0.8111 - loss: 0.4475 - val_accuracy: 0.8000 - val_auc: 0.7467 - val_loss: 0.4940
Epoch 6/400
7/7 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.8754 - auc: 0.8066 - loss: 0.4208 - val_accuracy: 0.8000 - val_auc: 0.7467 - val_loss: 0.4807
Epoch 7/400
7/7 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - accuracy: 0.8

5it [6:13:47, 4485.42s/it]

CPU times: user 12h 16min 20s, sys: 5h 10min 10s, total: 17h 26min 30s
Wall time: 6h 13min 47s


In [8]:
for i in range(NUM_EPOCHS):
    print(f"Epoch {(i + 1)}: TRAIN Accuracy = {np.round(epoch_acc[i], 3)} Loss = {np.round(epoch_loss[i], 3)} AUC = {np.round(epoch_auc[i], 3)}")
    print(f"Epoch {(i + 1)}: VAL Accuracy = {np.round(epoch_val_acc[i], 3)} Loss = {np.round(epoch_val_loss[i], 3)} AUC = {np.round(epoch_val_auc[i], 3)}")

with open("./logs/Liquidv3_expanded.txt", "w") as f:
    for i in range(NUM_EPOCHS):
        f.write(f"Epoch {(i + 1)}: TRAIN Accuracy = {np.round(epoch_acc[i], 3)} Loss = {np.round(epoch_loss[i], 3)} AUC = {np.round(epoch_auc[i], 3)}\n")
        f.write(f"Epoch {(i + 1)}: VAL Accuracy = {np.round(epoch_val_acc[i], 3)} Loss = {np.round(epoch_val_loss[i], 3)} AUC = {np.round(epoch_val_auc[i], 3)}\n")

Epoch 1: TRAIN Accuracy = 0.506 Loss = 0.691 AUC = 0.518
Epoch 1: VAL Accuracy = 0.613 Loss = 0.672 AUC = 0.735
Epoch 2: TRAIN Accuracy = 0.617 Loss = 0.664 AUC = 0.676
Epoch 2: VAL Accuracy = 0.627 Loss = 0.652 AUC = 0.729
Epoch 3: TRAIN Accuracy = 0.697 Loss = 0.645 AUC = 0.688
Epoch 3: VAL Accuracy = 0.74 Loss = 0.629 AUC = 0.76
Epoch 4: TRAIN Accuracy = 0.749 Loss = 0.622 AUC = 0.765
Epoch 4: VAL Accuracy = 0.82 Loss = 0.604 AUC = 0.772
Epoch 5: TRAIN Accuracy = 0.806 Loss = 0.595 AUC = 0.766
Epoch 5: VAL Accuracy = 0.82 Loss = 0.577 AUC = 0.77
Epoch 6: TRAIN Accuracy = 0.806 Loss = 0.572 AUC = 0.746
Epoch 6: VAL Accuracy = 0.82 Loss = 0.555 AUC = 0.762
Epoch 7: TRAIN Accuracy = 0.806 Loss = 0.552 AUC = 0.754
Epoch 7: VAL Accuracy = 0.82 Loss = 0.533 AUC = 0.788
Epoch 8: TRAIN Accuracy = 0.806 Loss = 0.534 AUC = 0.75
Epoch 8: VAL Accuracy = 0.82 Loss = 0.514 AUC = 0.771
Epoch 9: TRAIN Accuracy = 0.8 Loss = 0.515 AUC = 0.753
Epoch 9: VAL Accuracy = 0.82 Loss = 0.495 AUC = 0.786
Epoc